In [1]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
import pandas as pd
import footnet as ftnet

2023-11-03 13:17:05.594152: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-03 13:17:06.982995: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
import keras
import os
import tensorflow as tf
import numpy as np
import random

def reset_random_seeds(seed):
    os.environ['PYTHONHASHSEED']=str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    
reset_random_seeds(41)

In [3]:
csv_logger = CSVLogger( 'train_log.csv', append=True, separator=';')
earlystopper = EarlyStopping(patience=10, verbose=1)
checkpointer = ModelCheckpoint('Unet_checkpt_{val_r2_keras:.2f}.h5', 
                               verbose=1, save_best_only=True)

In [4]:
combdf = pd.read_csv('../combined_data_set_Sept11_20k.csv')
train_list = list(combdf[combdf['flag'] == 1]['path'])
valid_list = list(combdf[combdf['flag'] == 2]['path'])
test_list = list(combdf[combdf['flag'] == 3]['path'])
print(len(train_list), len(valid_list), len(test_list))

14449 2550 3001


In [5]:
train_DG = ftnet.data_generator_6h(train_list, batch_size=8)
valid_DG = ftnet.data_generator_6h(valid_list, batch_size=8)

In [6]:
model = ftnet.build_model()

2023-11-03 13:18:31.514759: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-03 13:18:35.287338: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13233 MB memory:  -> device: 0, name: NVIDIA A2, pci bus id: 0000:3b:00.0, compute capability: 8.6
/home/disk/atmos/taihe/anaconda3/envs/tf2/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
results = model.fit(train_DG, validation_data=valid_DG, epochs=1, shuffle=True, callbacks=[earlystopper, checkpointer, csv_logger])